In [38]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns

filtering positive and negative review and i am ignoring  sacore = 3 which is neutral score


In [61]:
# using the sqlite table to read data
con = sqlite3.connect('database.sqlite')

In [62]:
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 5000""", con) 


In [63]:
# if score is < 3 it mean -ve review and id score > 3 it mean +ve review
# and changing with 0 and 1 (+ve, -ve)
def partition(x):
    if x<3:
        return 0
    return 1

In [64]:
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [70]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

# below output shows the duplicate review - from same  user, text, userid helpfulnness and time for different productid


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


Data Cleaning

In [65]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [71]:
# first sorting the dataframe by ProductId
# removing duplicate from the dataframe 
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [74]:
sorted_data.head()
sorted_data.shape

(5000, 10)

In [73]:
final = sorted_data.drop_duplicates(subset={'UserId', 'ProfileName', "Time","Text"}, inplace=False)

In [76]:
final.shape # after removing duplicate the shape is

(4986, 10)

In [79]:
final['Score'].value_counts() # data contatin 4178 positive review and 808 -ve review

1    4178
0     808
Name: Score, dtype: int64

In [87]:
sent_0 = final['Text'].values[0]
sent_0

'Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.'

In [108]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from bs4 import BeautifulSoup
import re

In [119]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [146]:

filtered_sentence = []
for text in final['Text']:
    clean_text = []
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text()
    text = decontracted(text)

    text = re.sub(r"http\S+", "", text)
    text = re.sub("\S*\d\S*", "", text).strip()

    text = re.sub('[^A-Za-z0-9]+', ' ', text)


    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text.lower())
    for w in word_tokens:
        if w not in stop_words:
            clean_text.append(w)


    sentance = ' '.join(clean_text)
#     print(sentance)
    filtered_sentence.append(sentance)
#     print('===============================================')


In [147]:
len(filtered_sentence)

4986

In [176]:
filtered_sentence

['product available victor traps unreal course total fly genocide pretty stinky right nearby',
 'used victor fly bait seasons ca beat great product',
 'received shipment could hardly wait try product love slickers call instead stickers removed easily daughter designed signs printed reverse use car windows printed beautifully print shop program going lot fun product windows everywhere surfaces like tv screens computer monitors',
 'really good idea final product outstanding use decals car window everybody asks bought decals made two thumbs',
 'glad cocker standard poodle puppy loves stuff trust brand superior nutrition compare labels previous feed pedigree mostly corn little dude healthy happy high energy glossy coat also superior nutrition produces smaller compact stools',
 'using food months find excellent fact two dogs coton de tulear standard poodle puppy love food thriving coats excellent condition overall structure perfect good tasting dog good good deal owner around best food ever

In [150]:
clean_summary = []
summary_filtered_sentence = []
for text in final['Summary']:
    clean_text = []
    soup = BeautifulSoup(text, 'lxml')
    text = soup.get_text()
    text = decontracted(text)

    text = re.sub(r"http\S+", "", text)
    text = re.sub("\S*\d\S*", "", text).strip()

    text = re.sub('[^A-Za-z0-9]+', ' ', text)


    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text.lower())
    for w in word_tokens:
        if w not in stop_words:
            clean_summary.append(w)


    sentance = ' '.join(clean_summary)
#     print(sentance)
    summary_filtered_sentence.append(sentance)
#     print('===============================================')

In [151]:
len(summary_filtered_sentence)

4986

# Featurization

In [155]:
from sklearn.feature_extraction.text import CountVectorizer

In [164]:
filtered_sentence[0]

'product available victor traps unreal course total fly genocide pretty stinky right nearby'

In [212]:
count_vec = CountVectorizer()
count_vec.fit(filtered_sentence)
print('feature names are : ',count_vec.get_feature_names()[:50])
final_counts = count_vec.transform(filtered_sentence)
print(final_counts.toarray())
print(type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])


feature names are :  ['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability', 'able', 'abor', 'aboulutely', 'absence', 'absent', 'absoloutely', 'absolute', 'absolutely', 'absolutley', 'absolutly', 'absorb', 'absorbed', 'absorbs', 'abt', 'abundant', 'abuse', 'acacia', 'academy', 'acai', 'accelerated', 'acceleration', 'accept', 'acceptable', 'accepted', 'access', 'accessable', 'accessible', 'accessory', 'accident', 'accidentally', 'accidently', 'accidents', 'acclimatization', 'accolades', 'accommodate', 'accompanies', 'accompaniment', 'accompany', 'accompanying', 'accomplish']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
<class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (4986, 13099)
the number of unique words  13099


Bi-gram and n-gram

In [213]:
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
final_bigram_counts = count_vect.fit_transform(filtered_sentence)
print("the shape of out text BOW vectorizer ",final_bigram_counts.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_bigram_counts.get_shape()[1])

the shape of out text BOW vectorizer  (4986, 3005)
the number of unique words including both unigrams and bigrams  3005


TF-Idf

In [206]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


In [217]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(filtered_sentence)
final_tf_idf = tf_idf_vect.transform(filtered_sentence)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (4986, 3005)
the number of unique words including both unigrams and bigrams  3005
